In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:
# Load
df = pd.read_csv("5G_NIDD_multiclass_clean.csv", low_memory=False)

print("Original shape:", df.shape)

# Target
y = df['Label']
X = df.drop(columns=['Label', 'Attack Type', 'Attack Tool'], errors='ignore')

# Remove obvious non-learning columns
drop_cols = [
    'SrcMac','DstMac','SrcAddr','DstAddr','StartTime','LastTime',
    'SrcOui','DstOui'
]

X = X.drop(columns=[c for c in drop_cols if c in X.columns], errors='ignore')

# Keep only numeric features
X = X.select_dtypes(include=[np.number])

print("After numeric selection:", X.shape)


Original shape: (1215890, 112)
After numeric selection: (1215890, 86)


In [3]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)


In [4]:
selector = SelectKBest(score_func=f_classif, k=36)
X_selected = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 1  7 13 14 19 20 21 22 23 24 25 26 27 34 35 36 37 48 49 50 51 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 77 78 79 80] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Selected Features: ['Rank', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max', 'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts', 'DstPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcWin', 'DstWin', 'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']


In [5]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

num_classes = len(np.unique(y_encoded))
print("Classes:", num_classes)


Classes: 20


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)


In [7]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)   # FIT ONLY TRAIN
X_test  = scaler.transform(X_test)        # TRANSFORM TEST


In [8]:
X_train = X_train.reshape(-1, 36, 1)
X_test  = X_test.reshape(-1, 36, 1)


In [9]:
def MobileNetV1_BiGRU(drop_rate=0.5, gru_units=128, dense_units=256):

    inp = Input(shape=(36,1))

    # ----- CNN BRANCH -----
    x = Reshape((36,1,1))(inp)

    x = Conv2D(32,(3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(64,(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(128,(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    cnn_out = GlobalAveragePooling2D()(x)

    # ----- GRU BRANCH -----
    r = Bidirectional(GRU(gru_units, return_sequences=True))(inp)
    r = Bidirectional(GRU(gru_units))(r)

    # ----- FUSION -----
    merged = Concatenate()([cnn_out, r])

    merged = Dense(dense_units, activation="relu")(merged)
    merged = Dense(dense_units//2, activation="relu")(merged)
    merged = Dropout(drop_rate)(merged)

    out = Dense(num_classes, activation="softmax")(merged)

    model = Model(inp, out)
    return model


In [12]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.3 MB/s eta 0:00:00


In [13]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):

    model = MobileNetV1_BiGRU(
        drop_rate = hp.Choice("dropout",[0.3,0.5,0.6]),
        gru_units = hp.Choice("gru_units",[64,128,256]),
        dense_units = hp.Choice("dense",[128,256,512])
    )

    lr = hp.Choice("lr",[1e-2,1e-3,1e-4])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=7,   # normally 10
    factor=3,
    directory="tuning",
    project_name="5g_mobilenet_bigru"
)

# ---- Tune ONLY on subset ----
sample_idx = np.random.choice(len(X_train), size=int(len(X_train)*0.25), replace=False)
X_tune = X_train[sample_idx]
y_tune = y_train[sample_idx]

stop_early = EarlyStopping(monitor='val_loss', patience=3)

tuner.search(
    X_tune, y_tune,
    validation_split=0.2,
    epochs=10,
    batch_size=512,
    callbacks=[stop_early],
    verbose=1
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hps.values)


Trial 10 Complete [00h 02m 13s]
val_accuracy: 0.8548606038093567

Best val_accuracy So Far: 0.8548606038093567
Total elapsed time: 00h 19m 12s
Best Hyperparameters:
{'dropout': 0.5, 'gru_units': 128, 'dense': 256, 'lr': 0.001, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [14]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=512,
    callbacks=[
        EarlyStopping(patience=8, restore_best_weights=True),
        ReduceLROnPlateau(patience=4)
    ]
)


Epoch 1/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.7106 - loss: 0.7344 - val_accuracy: 0.8587 - val_loss: 0.3191 - learning_rate: 0.0010
Epoch 2/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 78s 46ms/step - accuracy: 0.8538 - loss: 0.3301 - val_accuracy: 0.8432 - val_loss: 0.4126 - learning_rate: 0.0010
Epoch 3/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 76s 44ms/step - accuracy: 0.8636 - loss: 0.3027 - val_accuracy: 0.8264 - val_loss: 0.6040 - learning_rate: 0.0010
Epoch 4/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 77s 45ms/step - accuracy: 0.8723 - loss: 0.2777 - val_accuracy: 0.8607 - val_loss: 0.3999 - learning_rate: 0.0010
Epoch 5/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 76s 45ms/step - accuracy: 0.8778 - loss: 0.2630 - val_accuracy: 0.8354 - val_loss: 0.4509 - learning_rate: 0.0010
Epoch 6/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 76s 44ms/step - accuracy: 0.8923 - loss: 0.2365 - val_accuracy: 0.8893 - val_loss: 0.2465 - learning_rate: 1.0000e-04
Epoch 7/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 76s 44ms/step - ac

In [15]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))


7600/7600 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     18761
           1       0.94      0.96      0.95     18730
           2       0.57      0.77      0.66      3656
           3       0.58      0.89      0.70      2890
           4       0.39      0.52      0.44       322
           5       0.34      0.16      0.22       271
           6       0.98      0.81      0.89      8927
           7       0.86      0.62      0.72      2822
           8       0.90      0.91      0.90      2290
           9       0.99      0.87      0.93      2305
          10       0.84      0.85      0.85      6203
          11       0.53      0.74      0.62      2531
          12       0.53      0.13      0.21      2329
          13       0.52      0.83      0.64      2331
          14       0.95      0.95      0.95      7624
          15       0.99      0.96      0.98      6334
          16       0.97      0.91    